In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive


In [ ]:

# General libraries
import os
import numpy as np
import pandas as pd 
import random
import cv2
import matplotlib.pyplot as plt
%matplotlib inline

# Deep learning libraries
import keras.backend as K
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Dropout, BatchNormalization
from keras.layers import Conv2D, SeparableConv2D, MaxPooling2D, LeakyReLU, Activation, Lambda, GlobalAveragePooling2D, DepthwiseConv2D, GlobalMaxPooling2D, Conv2DTranspose
from keras.layers import Add, Concatenate
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator
from keras.callbacks import ModelCheckpoint, ReduceLROnPlateau, EarlyStopping
from keras.optimizers import Adam, SGD

# Setting seeds for reproducibility
seed = 232
np.random.seed(seed)

Using TensorFlow backend.


In [ ]:
input_path = '/content/drive/My Drive/COVID DATA/fold1/' #You have to change Fold Name Here...................

In [ ]:
#Preprocessing and DataGenerator definition
#cond = 'COVID19'
class_name = ['/covid/', '/normal/', '/pneumonia/']
#class_name = [/normal/', '/pneumonia/']

def process_data(img_dims):

    X_train = []  
    y_train = []

    for cls in class_name:
      for img_name in (os.listdir(input_path + 'train' + cls)):
        img = cv2.imread(input_path + 'train' + cls + img_name, 0)
        try:
          img = cv2.resize(img, (img_dims, img_dims))
        except:
          print(img_name)
        X_train.append(img/255.0)

        if cls=='/covid/':
          y_train.append(0)
        elif cls=='/normal/':
          y_train.append(1)
        elif cls=='/pneumonia/':
          y_train.append(1)

    # I will be making predictions off of the test set in one batch size
    # This is useful to be able to get the confusion matrix
    X_test = []
    y_test = []

    for cls in class_name:
      for img_name in (os.listdir(input_path + 'test' + cls)):
          img = cv2.imread(input_path+'test' + cls + img_name, 0) #We are taking image in grayscale form. 
          try:
            img = cv2.resize(img, (img_dims, img_dims))
          except:
            print(img_name)
          X_test.append(img/255.0)

          if cls=='/covid/':
            y_test.append(0)
          elif cls=='/normal/':
            y_test.append(1)
          elif cls=='/pneumonia/':
            y_test.append(1)
        
    X_test = np.array(X_test)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    y_test = np.array(y_test)
    if len(np.unique(y_train))==2:
      y_train = y_train - 1
      y_test = y_test - 1
    
    return X_train, X_test, y_train, y_test

In [ ]:
# Hyperparameters
img_dims = 128
epochs = 15
batch_size = 32
X_train, X_test, y_train, y_test = process_data(img_dims)

In [ ]:
def create_block(input, chs, regularizer=False): ## Convolution block of 2 layers
    x = SeparableConv2D(chs, 3, padding="same")(input)
    x = Activation("relu")(x)
    x = BatchNormalization()(x)
    return x

def unet():  
    input = Input((img_dims,img_dims,1))
    
    # Encoder
    block1 = create_block(input, 32)
    x = MaxPooling2D(2)(block1)
    block2 = create_block(x, 64)
    x = MaxPooling2D(2)(block2)
    block3 = create_block(x, 128)
    x = MaxPooling2D(2)(block3)
    block4 = create_block(x, 256)
    
    # Middle
    x = MaxPooling2D(2)(block4)
    middle = create_block(x, 512)
    
    # Decoder
    x = Conv2DTranspose(256, kernel_size=2, strides=2)(middle)
    x = Concatenate()([block4, x])
    x = create_block(x, 128)
    x = Conv2DTranspose(64, kernel_size=2, strides=2)(x)
    x = Concatenate()([block3, x])
    x = create_block(x, 64)
    x = Conv2DTranspose(64, kernel_size=2, strides=2)(x)
    x = Concatenate()([block2, x])
    x = create_block(x, 64)
    x = Conv2DTranspose(32, kernel_size=2, strides=2)(x)
    x = Concatenate()([block1, x])
    x = create_block(x, 32)
   
    # output
    output = Conv2D(1, 1, activation='sigmoid', name='autoencoder_output')(x)

    middle_output = GlobalAveragePooling2D(name='concatenate1')(middle)
    y1 = Dense(512, activation='relu', name='concatenate2')(middle_output)
    y = Dense(2, activation='softmax', name='classification_output')(y1)
    
    return Model(input, [output, y]), Model(input, y1)


#encoder_model, autoencoder_model = unet()

model, model2 = unet()

losses = {'classification_output':'categorical_crossentropy', 'autoencoder_output':'mse'}
lossWeights = {'classification_output':1.0, 'autoencoder_output':1.0}


model.compile(optimizer='adam', loss=losses, loss_weights=lossWeights, metrics={'classification_output': 'accuracy', 'autoencoder_output': 'mse'})
#model.summary()

In [ ]:
X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], -1, 1)
X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], -1, 1)
print(X_train.shape, X_test.shape)
from keras.utils import to_categorical
y_train_cat = to_categorical(y_train)

(5012, 128, 128, 1) (1255, 128, 128, 1)


In [ ]:
model_name = 'MultiLoss_1.0Fold1' #Change it as necessary, This is the base name for model

weight_save_path = '/content/drive/My Drive/Results/Weights/' #..................................................Change path as necessary

lr_reduce = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=2, verbose=2, mode='auto', min_lr=0.000001)
#early_stop = EarlyStopping(monitor='val_loss', min_delta=0.001, patience=3, mode='min')
checkpoint = ModelCheckpoint(weight_save_path+model_name+'.h5', monitor='classification_output_accuracy', save_best_only=True, save_weights_only=True)

In [ ]:
# Fitting the model 
hist = model.fit(
           x=X_train, y={'classification_output':y_train_cat, 'autoencoder_output':X_train}, epochs=epochs, batch_size=batch_size, validation_split=0.1, callbacks=[lr_reduce, checkpoint])

In [ ]:
model.load_weights(weight_save_path+model_name+'.h5')

In [ ]:
from sklearn.metrics import classification_report

_,y_pred = model.predict(X_test)
y_pred_bool = np.argmax(y_pred, axis=1)

report = classification_report(y_test, y_pred_bool, output_dict=True)
print(classification_report(y_test, y_pred_bool))

result_df = report
result_df = pd.DataFrame(result_df).transpose()

print(result_df)

After this evaluation

In [ ]:

from tensorflow.keras.utils import plot_model

############## Make Sure to Change this ####################
plot_save_path = '/content/drive/My Drive/Results/ModelPlot/'#..................................................Change path as necessary
hist_save_path = '/content/drive/My Drive/Results/History/' #.................................................. Change path as necessary
result_save_path = '/content/drive/My Drive/Results/Result/' #..............................................Change path as necessary
confusion_matrix_save_path = '/content/drive/My Drive/Results/ConfusionMatrix/' #.............................Change path as necessary
############ According to the Path You are Using ##############


hist_df = pd.DataFrame(hist.history)
hist_csv_file = hist_save_path+model_name+'_history.csv'
with open(hist_csv_file, mode='w') as f:
    hist_df.to_csv(f)

plot_model(model, plot_save_path+model_name+'.png', show_shapes=True)

result_df = report
result_df = pd.DataFrame(result_df).transpose()

print(result_df)

result_df.to_csv(result_save_path+model_name+'_result.csv')

In [ ]:
from sklearn.metrics import accuracy_score, confusion_matrix
from seaborn import heatmap
from matplotlib import pyplot as plt

preds = np.argmax(y_pred, axis=1)

acc = accuracy_score(y_test, np.round(preds))*100
cm = confusion_matrix(y_test, np.round(preds))
cm_norm = confusion_matrix(y_test, np.round(preds), normalize='true')
#tn, fp, fn, tp = cm.ravel()

print('CONFUSION MATRIX ------------------')

ax = heatmap(cm, cmap='Accent', annot=True, xticklabels=['COVID19', 'NORMAL', 'PNEUMONIA'], yticklabels=['COVID19', 'NORMAL', 'PNEUMONIA'], square=True, fmt='d')

plt.savefig(confusion_matrix_save_path+model_name+'.png')
plt.show()
ax = heatmap(cm_norm, cmap='Accent', annot=True, xticklabels=['COVID19', 'NORMAL', 'PNEUMONIA'], yticklabels=['COVID19', 'NORMAL', 'PNEUMONIA'], square=True, fmt='f')

plt.savefig(confusion_matrix_save_path+model_name+'_normalized.png')